In [2]:
from torchvision.datasets import FashionMNIST

In [115]:
from torch.utils.data import DataLoader, TensorDataset
train = FashionMNIST('.', download=True, train=True)
dl = DataLoader(TensorDataset(train.train_data, train.train_labels), batch_size=int(len(train) * 0.3))
it = iter(dl)
valid = next(it)
valid_dl = DataLoader(TensorDataset(valid[0].type(torch.float32), valid[1]), batch_size=30)
Xs = []
ys = []
for i in it:
    Xs.append(i[0].type(torch.float32))
    ys.append(i[1])

train_X = torch.cat(Xs, 0)
    
train_dl = DataLoader(TensorDataset(train_X, torch.cat(ys, 0)), batch_size=30)
# valid[0].size(), valid[1].size()


In [137]:
import numpy as np
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

class FirstCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1)

    def forward(self, xb):
        xb = xb.view(-1, 1, 28, 28)
        xb = F.relu(self.conv1(xb))
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.conv3(xb))
        xb = F.avg_pool2d(xb, 4)
        return xb.view(-1, xb.size(1))

lr = 0.1

def loss_batch(model, loss_func, xb, yb, opt=None):
#     print(model(xb).type(), yb.type())
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

def accuracy(model, valid_dl):
    reals = []
    preds = []
    for xb, yb in valid_dl:
        reals.append(yb)
        preds.append(model(xb).argmax(dim=1))
    return accuracy_score(torch.cat(reals, 0), torch.cat(preds, 0))

def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(*[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl])
            val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

            print("Epoch %s: loss=%s, val_acc=%s, train_acc=%s" % (epoch, val_loss, accuracy(model, valid_dl), accuracy(model, train_dl)))


def initer(layer):
    if type(layer) == nn.Conv2d:
        nn.init.kaiming_normal_(layer.weight)
            
model = FirstCNN()
model.apply(initer)
fit(20, model, F.cross_entropy, torch.optim.Adam(model.parameters()), train_dl, valid_dl)
print("DON!")
#no momentum: loss=0.4727671396235625, val_acc=0.8349444444444445, train_acc=0.8478333333333333

Epoch 0: loss=0.739915046642224, val_acc=0.7484444444444445, train_acc=0.7522380952380953
Epoch 1: loss=0.6064661427090565, val_acc=0.7925555555555556, train_acc=0.800047619047619
Epoch 2: loss=0.5634652653709054, val_acc=0.8061666666666667, train_acc=0.8163095238095238
Epoch 3: loss=0.5187102157933017, val_acc=0.8207777777777778, train_acc=0.8318571428571429
Epoch 4: loss=0.4870860712788999, val_acc=0.8285555555555556, train_acc=0.8428571428571429
Epoch 5: loss=0.47746079829831917, val_acc=0.835, train_acc=0.8476904761904762
Epoch 6: loss=0.4547451528782646, val_acc=0.8442777777777778, train_acc=0.8593333333333333
Epoch 7: loss=0.4396409194916487, val_acc=0.8471111111111111, train_acc=0.8613095238095239
Epoch 8: loss=0.44191204976290466, val_acc=0.8442222222222222, train_acc=0.8602857142857143
Epoch 9: loss=0.44182077248891194, val_acc=0.8451111111111111, train_acc=0.8637142857142858
Epoch 10: loss=0.4391204577932755, val_acc=0.8473333333333334, train_acc=0.8673333333333333
Epoch 11: 

In [141]:
import numpy as np
from matplotlib.pyplot import imshow
#imshow(np.asarray(train[7][0]))
# train[7]
from torch.utils.data import TensorDataset
# train_ds = TensorDataset(train.train_data, train.train_labels
# TensorDataset(train)[:30000]
# [method_name for method_name in dir(train) if callable(getattr(train, method_name))]
# len(train)

In [37]:
import torch
from skorch import NeuralNetClassifier
net_regr = NeuralNetClassifier(
    module=FirstCNN,
    max_epochs=20,
    lr=0.1,
    criterion=torch.nn.NLLLoss,
#     criterion__weight=weight,
    optimizer=torch.optim.SGD,
    optimizer__momentum=0.9,
#     device='cuda',  # uncomment this to train with CUDA
)

In [30]:
train.train_labels.unique()

tensor([1, 9, 0, 3, 2, 7, 5, 6, 4, 8])

In [34]:
train.train_data = train.train_data.type('torch.FloatTensor')
net_regr.fit(train.train_data, train.train_labels)

  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1           inf       0.1000      -16.1242  15.5629
      2      -16.6445       0.1000      -17.0698  15.5948
      3      -17.3730       0.1004      -17.6433  15.3967
      4      -17.8578       0.1019      -18.0565  15.9598
      5      -18.2226       0.1047      -18.3799  15.4213
      6      -18.5153       0.1094      -18.6457  15.9111
      7      -18.7600       0.1146      -18.8713  15.4875
      8      -18.9701       0.1199      -19.0674  15.7156
      9      -19.1544       0.1279      -19.2407  15.3496
     10      -19.3184       0.1368      -19.3960  15.3957
     11      -19.4662       0.1431      -19.5367  15.3407
     12      -19.6007       0.1486      -19.6653  15.9689
     13      -19.7241       0.1552      -19.7838  15.3812
     14      -19.8381       0.1616      -19.8936  15.3919
     15      -19.9441       0.1651      -19.9959  15.3365
     16      -

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=FirstCNN(
    (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (conv3): Conv2d(16, 10, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  ),
)

In [138]:
test.test_data = test.test_data.type('torch.FloatTensor')
preds = model(test.test_data)

In [139]:
preds = preds.argmax(dim=1)

In [140]:
import pandas as pd
df = pd.DataFrame()
df['Class'] = preds
df.index.name = 'Id'
df.to_csv('submission.csv')
df

,Class
Id,
0,9
1,2
2,1
3,1
4,6
5,1
6,4
7,6
8,5
